**Installation des packages**

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tqdm import tqdm
import os
from io import BytesIO
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import keras

**Connection à Google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Copie de la base de donnée dans l'environnement Google Colab**

In [ ]:
import shutil

def copy_folder(source_folder, destination_folder):
    try:
        shutil.copytree(source_folder, destination_folder)
        print(f"Folder '{source_folder}' successfully copied to '{destination_folder}'.")
    except Exception as e:
        print(f"An error occurred: {e}")

source_folder_path = '/content/drive/MyDrive/dataset_myco_neg_x40_x1'
destination_folder_path = '/content/dataset_myco_x40_x1'

copy_folder(source_folder_path, destination_folder_path)


Folder '/content/drive/MyDrive/dataset_myco_neg_x40_x1' successfully copied to '/content/dataset_myco_x40_x1'.


**Extraction des images des albums pour les mettre dans leur classe respective**

Pour le Candida albicans

In [ ]:
import os
import shutil

source_folder = "/content/dataset_myco_x40_x1/C.albicans"

destination_folder = "/content/dataset_myco_x40_x1_1/C.albicans"

os.makedirs(destination_folder, exist_ok=True)

for root, _, files in os.walk(source_folder):
    for file_name in files:
        if file_name.endswith(('.jpg', '.jpeg', '.png', '.gif')):
            source_path = os.path.join(root, file_name)
            destination_path = os.path.join(destination_folder, file_name)
            shutil.copy2(source_path, destination_path)

print("Images extracted from source subfolders to destination folder.")

Images extracted from source subfolders to destination folder.


Pour le Candida glabrata

In [ ]:
import os
import shutil

source_folder = "/content/dataset_myco_x40_x1/C.glabrata"

destination_folder = "/content/dataset_myco_x40_x1_1/C.glabrata"

os.makedirs(destination_folder, exist_ok=True)

for root, _, files in os.walk(source_folder):
    for file_name in files:
        if file_name.endswith(('.jpg', '.jpeg', '.png', '.gif')):
            source_path = os.path.join(root, file_name)
            destination_path = os.path.join(destination_folder, file_name)
            shutil.copy2(source_path, destination_path)

print("Images extracted from source subfolders to destination folder.")

Images extracted from source subfolders to destination folder.


Pour le Candida parapsilosis

In [ ]:
import os
import shutil

source_folder = "/content/dataset_myco_x40_x1/C.parapsilosis"

destination_folder = "/content/dataset_myco_x40_x1_1/C.parapsilosis"

os.makedirs(destination_folder, exist_ok=True)

for root, _, files in os.walk(source_folder):
    for file_name in files:
        if file_name.endswith(('.jpg', '.jpeg', '.png', '.gif')):
            source_path = os.path.join(root, file_name)
            destination_path = os.path.join(destination_folder, file_name)
            shutil.copy2(source_path, destination_path)

print("Images extracted from source subfolders to destination folder.")

Images extracted from source subfolders to destination folder.


Pour les negatifs

In [ ]:
import os
import shutil

source_folder = "/content/dataset_myco_x40_x1/Negatif"

destination_folder = "/content/dataset_myco_x40_x1_1/Negatif"

os.makedirs(destination_folder, exist_ok=True)

for root, _, files in os.walk(source_folder):
    for file_name in files:
        if file_name.endswith(('.jpg', '.jpeg', '.png', '.gif')):
            source_path = os.path.join(root, file_name)
            destination_path = os.path.join(destination_folder, file_name)
            shutil.copy2(source_path, destination_path)

print("Images extracted from source subfolders to destination folder.")

**Split**

In [ ]:
!pip install split-folders
import splitfolders
input_folder = "/content/dataset_myco_x40_x1_1"
output_folder = "/content/dataset_myco_neg_output1"
train_folder = os.path.join(output_folder, "train")
val_folder = os.path.join(output_folder, "val")

splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(0.8, 0.2))

Copying files: 247 files [00:00, 445.14 files/s]


**Suppression des dossiers vide si besoin**

In [ ]:
def delete_empty_folders(directory):
    for root, dirs, files in os.walk(directory, topdown=False):
        for name in dirs:
            folder_path = os.path.join(root, name)
            if not os.listdir(folder_path):
                os.rmdir(folder_path)
                print(f"Deleted empty folder: {folder_path}")

# supprime sur le train:
directory_to_check = '/content/dataset_myco_output1/train'
delete_empty_folders(directory_to_check)

In [ ]:
#Supprime sur la validation
directory_path = '/content/dataset_myco_output1/val'

delete_empty_folders(directory_path)

**Compte le nombre d'images par classe pour l'entraînement et la validation**

In [ ]:
#####Pour compter les photos dans chaque train et val
def count_files(folder_path):
    count = 0
    for filename in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, filename)):
            count += 1
    return count

#####Pour train cal
folder_path = "/content/dataset_myco_neg_output1/train/C.albicans"
num_files = count_files(folder_path)
print("Number of files in the folder:", num_files)

Number of files in the folder: 80


In [ ]:
#####Pour train gla
folder_path = "/content/dataset_myco_neg_output1/train/C.glabrata"
num_files = count_files(folder_path)
print("Number of files in the folder:", num_files)

Number of files in the folder: 75


In [ ]:
#####Pour train parapsi
folder_path = "/content/dataset_myco_neg_output1/train/C.parapsilosis"
num_files = count_files(folder_path)
print("Number of files in the folder:", num_files)

Number of files in the folder: 42


In [ ]:
#####Pour val cal
folder_path = "/content/dataset_myco_neg_output1/val/C.albicans"
num_files = count_files(folder_path)
print("Number of files in the folder:", num_files)

Number of files in the folder: 20


In [ ]:
#####Pour val gla
folder_path = "/content/dataset_myco_neg_output1/val/C.glabrata"
num_files = count_files(folder_path)
print("Number of files in the folder:", num_files)

Number of files in the folder: 19


In [ ]:
#####Pour val parapsi
folder_path = "/content/dataset_myco_neg_output1/val/C.parapsilosis"
num_files = count_files(folder_path)
print("Number of files in the folder:", num_files)

Number of files in the folder: 11


***Data augmentation***

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/content/dataset_myco_neg_output1/train'
validation_data_dir = '/content/dataset_myco_neg_output1/val'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical')

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical')

Found 197 images belonging to 3 classes.
Found 50 images belonging to 3 classes.


Entrînement de l'InceptionV3

In [ ]:
from keras.applications import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

base_model = InceptionV3(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=13,
    validation_data=validation_generator,
    validation_steps=len(validation_generator))

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    '/content/dataset_myco/test',
    target_size=(299, 299),
    batch_size=10,
    class_mode='categorical')

test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')


87910968/87910968 [==============================] - 4s 0us/step
Epoch 1/13
7/7 [==============================] - 82s 11s/step - loss: 2.6865 - accuracy: 0.4264 - val_loss: 2.4696 - val_accuracy: 0.5200
Epoch 2/13
7/7 [==============================] - 71s 10s/step - loss: 1.3066 - accuracy: 0.5838 - val_loss: 0.8964 - val_accuracy: 0.5200
Epoch 3/13
7/7 [==============================] - 77s 11s/step - loss: 0.7392 - accuracy: 0.6396 - val_loss: 0.8098 - val_accuracy: 0.6200
Epoch 4/13
7/7 [==============================] - 77s 10s/step - loss: 0.5628 - accuracy: 0.7716 - val_loss: 0.6964 - val_accuracy: 0.7200
Epoch 5/13
7/7 [==============================] - 81s 11s/step - loss: 0.5462 - accuracy: 0.7766 - val_loss: 0.6255 - val_accuracy: 0.7000
Epoch 6/13
7/7 [==============================] - 81s 11s/step - loss: 0.4272 - accuracy: 0.8223 - val_loss: 0.6947 - val_accuracy: 0.7000
Epoch 7/13
7/7 [==============================] - 78s 11s/step - loss: 0.4245 - accuracy: 0.8274 - va

**Sauvegarde du modèle**

In [ ]:
model_path = '/content/yeast_species_classifier_x40_x1.h5'

model.save(model_path)

print(f"Model saved at: {model_path}")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved at: /content/yeast_species_classifier_x40_x1.h5


**Métriques sur le jeu de données test**

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    '/content/test_x40_x1',
    target_size=(299, 299),
    batch_size=20,
    class_mode='categorical',
    shuffle=False)


model= tf.keras.models.load_model('/content/yeast_species_classifier_x40_x1.h5')

predictions = model.predict(test_generator)

predicted_classes = np.argmax(predictions, axis=1)

true_classes = test_generator.classes

class_labels = list(test_generator.class_indices.keys())

report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print("Classification Report:\n", report)

conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:\n", conf_matrix)

precision = np.diag(conf_matrix) / np.sum(conf_matrix, axis=0)
recall = np.diag(conf_matrix) / np.sum(conf_matrix, axis=1)
f1_score = 2 * (precision * recall) / (precision + recall)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)

Found 124 images belonging to 4 classes.
7/7 [==============================] - 45s 6s/step
Classification Report:
                 precision    recall  f1-score   support

    C.albicans       0.15      0.06      0.09        31
    C.glabrata       0.83      0.87      0.85        60
C.parapsilosis       0.29      0.70      0.41        20
       Negatif       0.00      0.00      0.00        13

      accuracy                           0.55       124
     macro avg       0.32      0.41      0.34       124
  weighted avg       0.48      0.55      0.50       124

Confusion Matrix:
 [[ 2  3 26  0]
 [ 0 52  8  0]
 [ 0  6 14  0]
 [11  2  0  0]]
Precision: [0.15384615 0.82539683 0.29166667        nan]
Recall: [0.06451613 0.86666667 0.7        0.        ]
F1 Score: [0.09090909 0.84552846 0.41176471        nan]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-21-96410fdf5d15>:36: RuntimeWarning: invalid valu

**Prédiction sur une image**

In [ ]:
import numpy as np
from keras.models import load_model
from keras.preprocessing import image

model = load_model('/content/yeast_species_classifier_x40_x1.h5')

img_path = '/content/IMG_4527.jpg'

class_labels = ['C.albicans', 'C.glabrata', 'C.parapsilosis', 'Negatif']

img = image.load_img(img_path, target_size=(299, 299))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0

prediction = model.predict(img_array)

predicted_class_index = np.argmax(prediction)

actual_class_label = class_labels[predicted_class_index]

confidence_score = np.max(prediction)

print("Prediction:")
print(f"Class: {actual_class_label}")
print(f"Confidence Score: {confidence_score}")
print(f"Class Probabilities: {prediction}")
